In [1]:
import os

midi_count = 0

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.lower().endswith('.mid'):
            midi_count += 1

print(f"Total number of MIDI files: {midi_count}")


Total number of MIDI files: 3896


In [2]:

!pip install music21 torch pretty_midi tqdm --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.1 MB/s eta 0:00:00


In [3]:

import os
import music21
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pretty_midi
import pickle
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import random
from tqdm import tqdm


In [4]:
midi_dir = Path("/kaggle/input/nintendo-midis/Nintendo")
chord_vocab, melody_vocab = {}, {}
chord_seqs, melody_seqs = [], []

max_len = 64
max_files = 4000
midi_files = list(midi_dir.rglob("*.mid"))
random.shuffle(midi_files)


def note_to_int(note):
    return int(note.pitch.midi)

def chord_to_label(chord):
    root = chord.root().name
    quality = chord.quality
    return f"{root}_{quality}"

for file in tqdm(midi_files[:max_files], desc="Processing MIDI files"):
    try:
        score = music21.converter.parse(file)
        if isinstance(score, music21.stream.Opus):
            if len(score.scores) == 0:
                continue
            score = score.scores[0]

        chords = score.chordify().flatten().getElementsByClass('Chord')
        melody = score.parts[0].flatten().getElementsByClass('Note')

        chord_seq = []
        for c in chords[:max_len]:
            label = chord_to_label(c)
            if label not in chord_vocab:
                chord_vocab[label] = len(chord_vocab)
            chord_seq.append(chord_vocab[label])

        melody_seq = []
        for n in melody[:max_len]:
            midi = note_to_int(n)
            if midi not in melody_vocab:
                melody_vocab[midi] = len(melody_vocab)
            melody_seq.append(melody_vocab[midi])

        if len(chord_seq) == len(melody_seq) == max_len:
            chord_seqs.append(chord_seq)
            melody_seqs.append(melody_seq)
    except Exception as e:
        print(f"Failed on {file}: {e}")

with open("processed.pkl", "wb") as f:
    pickle.dump({
        "chord_seqs": chord_seqs,
        "melody_seqs": melody_seqs,
        "chord_vocab": chord_vocab,
        "melody_vocab": melody_vocab
    }, f)

print(f"Saved Nintendo MIDI sequences.")


Processing MIDI files:  86%|████████▌ | 3347/3896 [1:03:55<06:00,  1.52it/s] 

Failed on /kaggle/input/nintendo-midis/Nintendo/3DS/FireEmblemAwakening/.mid: cannot find a format extensions for: /kaggle/input/nintendo-midis/Nintendo/3DS/FireEmblemAwakening/.mid


Processing MIDI files: 100%|██████████| 3896/3896 [1:14:50<00:00,  1.15s/it]  

Saved Nintendo MIDI sequences.


In [5]:
class ChordMelodyDataset(Dataset):
    def __init__(self, X, Y, seq_len):
        self.X = X
        self.Y = Y
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx][:self.seq_len]),
            torch.tensor(self.Y[idx][:self.seq_len])
        )

class ChordToMelodyLSTM(nn.Module):
    def __init__(self, chord_vocab, melody_vocab, hidden_dim=128):
        super().__init__()
        self.chord_embed = nn.Embedding(chord_vocab, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
        self.attn = nn.MultiheadAttention(embed_dim=hidden_dim*2, num_heads=2, batch_first=True)
        self.out = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim*2, melody_vocab)
        )

    def forward(self, chords):
        x = self.chord_embed(chords)
        h, _ = self.lstm(x)
        attn_output, _ = self.attn(h, h, h)
        return self.out(attn_output)

In [8]:
with open("processed.pkl", "rb") as f:
    data = pickle.load(f)

chord_seqs = data["chord_seqs"]
melody_seqs = data["melody_seqs"]
chord_vocab_size = len(data["chord_vocab"])
melody_vocab_size = len(data["melody_vocab"])

seq_len = 64

filtered_pairs = [
    (c, m) for c, m in zip(chord_seqs, melody_seqs)
    if len(c) == seq_len and len(m) == seq_len
]

chord_seqs, melody_seqs = zip(*filtered_pairs) if filtered_pairs else ([], [])

dataset = ChordMelodyDataset(chord_seqs, melody_seqs, seq_len)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = ChordToMelodyLSTM(chord_vocab_size, melody_vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):
    total_loss = 0
    for chords, melody in loader:
        logits = model(chords)
        loss = criterion(logits.view(-1, melody_vocab_size), melody.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}: Loss = {total_loss:.4f}")

torch.save(model.state_dict(), "lstm_model.pt")
print("Model saved!")

Epoch 0: Loss = 406.9728
Epoch 1: Loss = 389.3077
Epoch 2: Loss = 384.3761
Epoch 3: Loss = 381.5696
Epoch 4: Loss = 379.4148
Epoch 5: Loss = 377.0618
Epoch 6: Loss = 375.1254
Epoch 7: Loss = 373.1363
Epoch 8: Loss = 370.9506
Epoch 9: Loss = 368.5528
Epoch 10: Loss = 366.4501
Epoch 11: Loss = 363.3060
Epoch 12: Loss = 360.6589
Epoch 13: Loss = 356.9566
Epoch 14: Loss = 353.0570
Epoch 15: Loss = 350.1042
Epoch 16: Loss = 345.1470
Epoch 17: Loss = 340.0254
Epoch 18: Loss = 334.5153
Epoch 19: Loss = 329.3661
Epoch 20: Loss = 324.8503
Epoch 21: Loss = 320.3435
Epoch 22: Loss = 316.0900
Epoch 23: Loss = 311.9285
Epoch 24: Loss = 307.3275
Epoch 25: Loss = 304.0875
Epoch 26: Loss = 301.2781
Epoch 27: Loss = 298.1860
Epoch 28: Loss = 295.4268
Epoch 29: Loss = 293.0991
Epoch 30: Loss = 291.3836
Epoch 31: Loss = 288.7888
Epoch 32: Loss = 287.0538
Epoch 33: Loss = 285.4287
Epoch 34: Loss = 283.1119
Epoch 35: Loss = 282.0360
Epoch 36: Loss = 280.5579
Epoch 37: Loss = 278.5476
Epoch 38: Loss = 276.7

In [9]:
def generate_midi(melody_ids, melody_vocab, filename="generated.mid"):
    inv_vocab = {v: k for k, v in melody_vocab.items()}
    pm = pretty_midi.PrettyMIDI()
    inst = pretty_midi.Instrument(program=0)
    for i, idx in enumerate(melody_ids):
        pitch = inv_vocab.get(idx, 60)
        note = pretty_midi.Note(
            velocity=100, pitch=pitch, start=i*0.5, end=(i+1)*0.5
        )
        inst.notes.append(note)
    pm.instruments.append(inst)
    pm.write(filename)


In [10]:
def sample_logits(logits, temperature=1.0):
    probs = torch.softmax(logits / temperature, dim=-1)
    return torch.multinomial(probs, num_samples=1)

def generate_from_model(model, chord_sequence, melody_vocab, device="cpu", temperature=1.0):
    model.eval()
    with torch.no_grad():
        chords = torch.tensor(chord_sequence).unsqueeze(0).to(device)
        logits = model(chords).squeeze(0)
        sampled = [sample_logits(logits[i], temperature).item() for i in range(logits.size(0))]
        return sampled

model.load_state_dict(torch.load("lstm_model.pt"))
model.eval()

nintendo_intro = [data["chord_vocab"].get(label) for label in [
    "C_major", "F_major", "G_major", "C_major"
]]


nintendo_intro = [c if c is not None else 0 for c in nintendo_intro] + [0] * (64 - 16)

generated_ids = generate_from_model(model, nintendo_intro, data["melody_vocab"], temperature=0.8)
generate_midi(generated_ids, data["melody_vocab"], "generate_final_res.mid")
